In [49]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
import data_processing as dp
from scipy import signal
from scipy.stats import skew
import seaborn as sns
import matplotlib.pyplot as plt
import pywt
import os
import time
import h5py
from itertools import combinations

In [2]:
for dirname, _, filenames in os.walk('.\data'):
    for filename in filenames:
        print(os.path.join(dirname,filename))

.\data\G04_FoG_trial_1_emg.csv
.\data\G04_FoG_trial_2_emg.csv
.\data\G06_FoG_trial_1_emg.csv
.\data\G06_FoG_trial_2_emg.csv
.\data\G06_FoG_trial_3_emg.csv
.\data\G07_Freezing_Trial1_trial_1_emg.csv
.\data\G08_FoG_1_trial_1_emg.csv
.\data\G08_FoG_2_trial_1_emg.csv
.\data\G11_FoG_trial_1_emg.csv
.\data\G11_FoG_trial_2_emg.csv
.\data\P379_M050_2_OFF_A_FoG_trial_1_emg.csv
.\data\P379_M050_2_OFF_A_FoG_trial_2_emg.csv
.\data\P379_M050_2_OFF_A_FoG_trial_3_emg.csv
.\data\P379_M050_2_OFF_B_FoG_trial_1_emg.csv
.\data\P379_M050_2_OFF_B_FoG_trial_2_emg.csv
.\data\P379_M050_2_OFF_B_FoG_trial_3_emg.csv
.\data\P551_M050_2_A_FoG_trial_1_emg.csv
.\data\P551_M050_2_B_FoG_trial_1_emg.csv
.\data\P551_M050_2_B_FoG_trial_2_emg.csv
.\data\P812_M050_2_B_FoG_trial_1_emg.csv
.\data\P812_M050_2_B_FoG_trial_2_emg.csv
.\data\其他\labels.txt
.\data\其他\P812_M050_2_B_FoG_trial1_annotation.csv
.\data\其他\P812_M050_2_B_FoG_trials.mat
.\data\其他\P812_M050_2_B_FoG_trial_1_out_left_foot.csv
.\data\其他\P812_M050_2_B_FoG_trial_1

In [72]:
path2 = '.\data\正常\G11_Walking_trial_4_emg.csv'

In [6]:
Data = pd.read_csv('./processed data/dataframe_W256_S64.csv')

# read file name of data with various Labels
df = pd.read_csv('./useful_data_label.csv',index_col=0) 
drop = 'P551_M050_2_B_FoG_trial_2_emg.csv'
ind_drop = df.columns!=drop
# read file name of data with only label 0
df2 = pd.read_csv('./unuseful_data_label.csv',index_col=0)
# read some of the data with only label 0
df3 = pd.read_csv('./data/file_name.txt',header=None)

files = np.concatenate([np.array(df.columns),np.array(df3.loc[:,0])])
ind = Data.File.isin(files)
Data_sel = Data[ind]
Data_rest = Data[~ind]

In [7]:
feature_col = ['_IEMG','_MAV','_SSI','_VAR','_RMS',
               '_WL','_ZC','_SSC','_WAMP','_skew','_Acti','_AR','_HIST','_MDF']

feature_all = Data_sel.iloc[:,1:-1]
ind_temp1 = feature_all.columns.str.contains('MAV')
ind_temp2 = feature_all.columns.str.contains('RMS')
ind_temp3 = feature_all.columns.str.contains('VAR')
ind_temp = ind_temp1|ind_temp2|ind_temp3
feature = feature_all.loc[:,~ind_temp]
y = Data_sel.iloc[:,0]
feature2_all = Data_rest.iloc[:,1:-1]
feature2 = feature2_all.loc[:,~ind_temp]
y2 = Data_rest.iloc[:,0]

In [4]:
path = './processed data/data_set_after_window_withoutSC.hdf5'
with h5py.File(path,'r') as f:
    x = f['cwt_data'][...]
    y = f['label2'][...]

In [211]:
import imp
imp.reload(dp)

<module 'data_processing' from 'E:\\Document\\jupyter\\Master Thesis\\data_processing.py'>

In [66]:
import ctypes
player = ctypes.windll.kernel32
player.Beep(1000,200)

1

In [8]:
width = 256
threshold_WAMP = 30
threshold_ZC = 0
threshold_SSC = 1
bins=9
bound = 70
HIST_range = (-bound,bound)

In [127]:
path = './processed data/data_set_after_window_S64_withoutSC_allPa.hdf5'
with h5py.File(path,'r') as f:
    x = f['cwt_data'][...]
    y = f['label2'][...]
feature = dp.generate_feature(x,threshold_WAMP=threshold_WAMP,
                              threshold_ZC=threshold_ZC,
                              threshold_SSC=threshold_SSC,
                              bins=bins,ranges=HIST_range)
#feature2 = dp.generate_feature(x2)
player.Beep(1000,200)

threshold_WAMP:30.0, threshold_ZC:0.0, threshold_SSC:1.0,bins:9,ranges:(-70,70)
IEMG,MAV,SSI,VAR,RMS,WL,ZC,SSC,WAMP,skew,Acti,AR,HIST


1

In [1]:
with h5py.File('./processed data/nfeatures_W256_S64_WAMP30.hdf5','r') as f:
    feature = f['features'][...]
    y = f['labels'][...]

NameError: name 'h5py' is not defined

In [ ]:
feature2,y2 = dp.pipeline_feature(path2,width=256,stride=64,
                                  scaler=False,
                                  threshold_WAMP=threshold_WAMP,
                                  threshold_ZC=threshold_ZC,
                                  threshold_SSC=threshold_SSC,
                                  bins=bins,ranges=HIST_range)

In [222]:
with h5py.File('./processed data/nfeatures_rest_W256_S64.hdf5','r') as f:
    feature2 = f['features'][...]
    y2 = f['labels'][...]

In [223]:
sc = StandardScaler(with_mean=True)
feature_sc = sc.fit_transform(feature)
feature2_sc = sc.transform(feature2)

# Model

In [9]:
from tensorflow.keras import layers,Model,callbacks
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder

In [58]:
def train_model(model,feature,y,file=None):
    ind = ((y==4)|(y==1)|(y==2)|(y==3)|(y==6))
    y_01 = y.copy()
    y_01[ind] = 1
    x_full,x_test,y_full,y_test = train_test_split(feature,y_01,
                                                   test_size=0.2,
                                                   random_state=123,
                                                   shuffle=False)
    x_train,x_valid,y_train,y_valid = train_test_split(x_full,y_full,
                                                       test_size=0.2,
                                                       random_state=555,
                                                       shuffle=True)
    
    
    early_stopping = callbacks.EarlyStopping(patience = 20,
                                             monitor = 'val_accuracy', 
                                             restore_best_weights=True)
    history = model.fit(x_train,y_train,validation_data=[x_valid,y_valid],
                        epochs=200,batch_size=50,
                        callbacks=[early_stopping])
    y_pred_t=model.predict(x_test)
    test = metrics.accuracy_score(y_test,y_pred_t>0.5)
    y_pred_v=model.predict(x_valid)
    valid = metrics.accuracy_score(y_valid,y_pred_v>0.5)
    y_pred_ta=model.predict(x_train)
    train = metrics.accuracy_score(y_train,y_pred_ta>0.5)
    print('test:%f'%test)
    print('valid:%f'%valid)
    print('train:%f'%train)
    if file != None:
        model.save_model(file)
    return train,valid,test

def test_model(model,feature,y):
    ind = ((y==4)|(y==1)|(y==2)|(y==3)|(y==6))
    y_01 = y.copy()
    y_01[ind] = 1
    y_pred=model.predict(feature)
    test = metrics.accuracy_score(y_01,y_pred>0.5)
    print('acc:%f'%test)
    return test

## ANN

In [224]:
#ind = ((y==1)|(y==2)|(y==3)|(y==6))
ind = ((y==4)|(y==1)|(y==2)|(y==3)|(y==6))
ind_f = [0,1,6,42,46,57,62]
#y_01 = y[ind].copy()
#y_01[y_01==1]=0
#y_01[y_01==2]=1
#y_01[y_01==3]=2
#y_01[y_01==6]=3
y_01 = y.copy()
y_01[ind] = 1
oh_ec = OneHotEncoder()
y_oh = oh_ec.fit_transform(y_01[:,np.newaxis]).toarray()
x_full,x_test,y_full,y_test = train_test_split(feature_sc,y_01,test_size=0.2,random_state=123,shuffle=False)
x_train,x_valid,y_train,y_valid = train_test_split(x_full,y_full,test_size=0.2,random_state=555,shuffle=True)

In [75]:
input_ = layers.Input(shape=feature_sc.shape[1:])
l1 = layers.Dense(128,activation='elu')(input_)
drop1 = layers.Dropout(0.2)(l1)
l2 = layers.Dense(64,activation='elu')(drop1)
drop2 = layers.Dropout(0.2)(l2)
l3 = layers.Dense(32,activation='elu')(drop2)
drop3 = layers.Dropout(0.2)(l3)
l4 = layers.Dense(16,activation='elu')(drop3)
drop4 = layers.Dropout(0.2)(l4)
output = layers.Dense(1,activation='sigmoid')(drop4)
model = Model(inputs=[input_],outputs=[output])

In [76]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
early_stopping = callbacks.EarlyStopping(patience = 20,
                                         monitor = 'val_accuracy', 
                                         restore_best_weights=True)
history = model.fit(x_train,y_train,validation_data=[x_valid,y_valid],
                    epochs=200,batch_size=50,
                   callbacks=[early_stopping])

# Train model

In [33]:
sc = StandardScaler(with_mean=True)
feature_sc = sc.fit_transform(feature)
feature2_sc = sc.transform(feature2)
train_model(model,feature_sc,np.array(y))

Train on 22000 samples, validate on 5500 samples
Epoch 1/200
22000/22000 [==============================] - 2s 81us/sample - loss: 0.3253 - accuracy: 0.8672 - val_loss: 0.2081 - val_accuracy: 0.9209
Epoch 2/200
22000/22000 [==============================] - 1s 50us/sample - loss: 0.2399 - accuracy: 0.9108 - val_loss: 0.1831 - val_accuracy: 0.9316
Epoch 3/200
22000/22000 [==============================] - 1s 51us/sample - loss: 0.2167 - accuracy: 0.9188 - val_loss: 0.1717 - val_accuracy: 0.9336
Epoch 4/200
22000/22000 [==============================] - 1s 67us/sample - loss: 0.1985 - accuracy: 0.9273 - val_loss: 0.1679 - val_accuracy: 0.9375
Epoch 5/200
22000/22000 [==============================] - 2s 86us/sample - loss: 0.1856 - accuracy: 0.9334 - val_loss: 0.1559 - val_accuracy: 0.9413
Epoch 6/200
22000/22000 [==============================] - 2s 68us/sample - loss: 0.1728 - accuracy: 0.9350 - val_loss: 0.1627 - val_accuracy: 0.9404
Epoch 7/200
22000/22000 [==========================

Epoch 55/200
22000/22000 [==============================] - 1s 54us/sample - loss: 0.0470 - accuracy: 0.9831 - val_loss: 0.0987 - val_accuracy: 0.9711
Epoch 56/200
22000/22000 [==============================] - 1s 48us/sample - loss: 0.0454 - accuracy: 0.9846 - val_loss: 0.1004 - val_accuracy: 0.9695
Epoch 57/200
22000/22000 [==============================] - 1s 48us/sample - loss: 0.0421 - accuracy: 0.9840 - val_loss: 0.0979 - val_accuracy: 0.9684
Epoch 58/200
22000/22000 [==============================] - 1s 54us/sample - loss: 0.0417 - accuracy: 0.9856 - val_loss: 0.0990 - val_accuracy: 0.9693
Epoch 59/200
22000/22000 [==============================] - 1s 48us/sample - loss: 0.0447 - accuracy: 0.9842 - val_loss: 0.0931 - val_accuracy: 0.9727
Epoch 60/200
22000/22000 [==============================] - 1s 49us/sample - loss: 0.0440 - accuracy: 0.9837 - val_loss: 0.0964 - val_accuracy: 0.9711
Epoch 61/200
22000/22000 [==============================] - 1s 48us/sample - loss: 0.0411 - ac

22000/22000 [==============================] - 1s 52us/sample - loss: 0.0251 - accuracy: 0.9912 - val_loss: 0.0939 - val_accuracy: 0.9755
Epoch 110/200
22000/22000 [==============================] - 1s 51us/sample - loss: 0.0237 - accuracy: 0.9918 - val_loss: 0.0954 - val_accuracy: 0.9744
Epoch 111/200
22000/22000 [==============================] - 1s 49us/sample - loss: 0.0253 - accuracy: 0.9911 - val_loss: 0.0980 - val_accuracy: 0.9769
Epoch 112/200
22000/22000 [==============================] - 1s 56us/sample - loss: 0.0249 - accuracy: 0.9915 - val_loss: 0.1048 - val_accuracy: 0.9736
Epoch 113/200
22000/22000 [==============================] - 1s 50us/sample - loss: 0.0234 - accuracy: 0.9921 - val_loss: 0.1058 - val_accuracy: 0.9760
Epoch 114/200
22000/22000 [==============================] - 1s 50us/sample - loss: 0.0247 - accuracy: 0.9915 - val_loss: 0.0907 - val_accuracy: 0.9762
Epoch 115/200
22000/22000 [==============================] - 1s 50us/sample - loss: 0.0243 - accuracy:

(0.9994545454545455, 0.976909090909091, 0.9460363636363637)

In [34]:
test_model(model,feature2_sc,np.array(y2))

acc:0.961448


0.9614476143529074

In [30]:
[0.9988181818181818, 0.9772727272727273, 0.9345454545454546, 0.9541356008790848]

[0.9988181818181818,
 0.9772727272727273,
 0.9345454545454546,
 0.9541356008790848]

# test feature from Left or Right

In [77]:
acc={}
cols = ['LEFT','RIGHT']

sc = StandardScaler(with_mean=True)

for col in cols:
    ind_temp=feature.columns.str.contains(col)
    feature_sc = sc.fit_transform(feature.loc[:,ind_temp])
    feature2_sc = sc.transform(feature2.loc[:,ind_temp])
    train,valid,test = train_model(model,feature_sc,np.array(y))
    
    acc_rest=test_model(model,feature2_sc,np.array(y2))
    acc[col] = [train,valid,test,acc_rest]
    print(acc_rest)

Train on 22000 samples, validate on 5500 samples
Epoch 1/200
22000/22000 [==============================] - 3s 123us/sample - loss: 0.4044 - accuracy: 0.8259 - val_loss: 0.2862 - val_accuracy: 0.8849
Epoch 2/200
22000/22000 [==============================] - 2s 78us/sample - loss: 0.3294 - accuracy: 0.8668 - val_loss: 0.2688 - val_accuracy: 0.8893
Epoch 3/200
22000/22000 [==============================] - 1s 66us/sample - loss: 0.3026 - accuracy: 0.8780 - val_loss: 0.2530 - val_accuracy: 0.8989
Epoch 4/200
22000/22000 [==============================] - 1s 66us/sample - loss: 0.2861 - accuracy: 0.8882 - val_loss: 0.2438 - val_accuracy: 0.8989
Epoch 5/200
22000/22000 [==============================] - 1s 62us/sample - loss: 0.2742 - accuracy: 0.8920 - val_loss: 0.2347 - val_accuracy: 0.9082
Epoch 6/200
22000/22000 [==============================] - 2s 69us/sample - loss: 0.2664 - accuracy: 0.8972 - val_loss: 0.2278 - val_accuracy: 0.9096
Epoch 7/200
22000/22000 [=========================

Epoch 55/200
22000/22000 [==============================] - 1s 64us/sample - loss: 0.1462 - accuracy: 0.9473 - val_loss: 0.1581 - val_accuracy: 0.9440
Epoch 56/200
22000/22000 [==============================] - 1s 57us/sample - loss: 0.1454 - accuracy: 0.9445 - val_loss: 0.1578 - val_accuracy: 0.9425
Epoch 57/200
22000/22000 [==============================] - 2s 73us/sample - loss: 0.1445 - accuracy: 0.9465 - val_loss: 0.1574 - val_accuracy: 0.9387
Epoch 58/200
22000/22000 [==============================] - 1s 63us/sample - loss: 0.1422 - accuracy: 0.9450 - val_loss: 0.1562 - val_accuracy: 0.9436
Epoch 59/200
22000/22000 [==============================] - 1s 65us/sample - loss: 0.1419 - accuracy: 0.9459 - val_loss: 0.1538 - val_accuracy: 0.9402
Epoch 60/200
22000/22000 [==============================] - 1s 56us/sample - loss: 0.1410 - accuracy: 0.9460 - val_loss: 0.1614 - val_accuracy: 0.9407
Epoch 61/200
22000/22000 [==============================] - 1s 52us/sample - loss: 0.1388 - ac

22000/22000 [==============================] - 1s 50us/sample - loss: 0.1017 - accuracy: 0.9626 - val_loss: 0.1405 - val_accuracy: 0.9509
Epoch 110/200
22000/22000 [==============================] - 2s 69us/sample - loss: 0.1046 - accuracy: 0.9614 - val_loss: 0.1409 - val_accuracy: 0.9498
Epoch 111/200
22000/22000 [==============================] - 1s 67us/sample - loss: 0.1072 - accuracy: 0.9594 - val_loss: 0.1392 - val_accuracy: 0.9507
Epoch 112/200
22000/22000 [==============================] - 1s 50us/sample - loss: 0.1034 - accuracy: 0.9610 - val_loss: 0.1404 - val_accuracy: 0.9493
Epoch 113/200
22000/22000 [==============================] - 1s 48us/sample - loss: 0.1070 - accuracy: 0.9616 - val_loss: 0.1348 - val_accuracy: 0.9505
Epoch 114/200
22000/22000 [==============================] - 1s 50us/sample - loss: 0.1050 - accuracy: 0.9626 - val_loss: 0.1385 - val_accuracy: 0.9515
Epoch 115/200
22000/22000 [==============================] - 1s 51us/sample - loss: 0.1052 - accuracy:

Epoch 163/200
22000/22000 [==============================] - 1s 47us/sample - loss: 0.0850 - accuracy: 0.9685 - val_loss: 0.1321 - val_accuracy: 0.9571
Epoch 164/200
22000/22000 [==============================] - 1s 54us/sample - loss: 0.0849 - accuracy: 0.9688 - val_loss: 0.1319 - val_accuracy: 0.9569
Epoch 165/200
22000/22000 [==============================] - 1s 57us/sample - loss: 0.0853 - accuracy: 0.9690 - val_loss: 0.1291 - val_accuracy: 0.9558
Epoch 166/200
22000/22000 [==============================] - 1s 45us/sample - loss: 0.0906 - accuracy: 0.9682 - val_loss: 0.1269 - val_accuracy: 0.9553
Epoch 167/200
22000/22000 [==============================] - 1s 46us/sample - loss: 0.0836 - accuracy: 0.9693 - val_loss: 0.1415 - val_accuracy: 0.9524
Epoch 168/200
22000/22000 [==============================] - 1s 54us/sample - loss: 0.0859 - accuracy: 0.9685 - val_loss: 0.1258 - val_accuracy: 0.9527
Epoch 169/200
22000/22000 [==============================] - 2s 73us/sample - loss: 0.08

22000/22000 [==============================] - 1s 48us/sample - loss: 0.1571 - accuracy: 0.9427 - val_loss: 0.1736 - val_accuracy: 0.9320
Epoch 34/200
22000/22000 [==============================] - 1s 45us/sample - loss: 0.1559 - accuracy: 0.9409 - val_loss: 0.1750 - val_accuracy: 0.9336
Epoch 35/200
22000/22000 [==============================] - 1s 51us/sample - loss: 0.1519 - accuracy: 0.9438 - val_loss: 0.1748 - val_accuracy: 0.9298
Epoch 36/200
22000/22000 [==============================] - 1s 49us/sample - loss: 0.1522 - accuracy: 0.9416 - val_loss: 0.1662 - val_accuracy: 0.9345
Epoch 37/200
22000/22000 [==============================] - 1s 47us/sample - loss: 0.1525 - accuracy: 0.9430 - val_loss: 0.1668 - val_accuracy: 0.9342
Epoch 38/200
22000/22000 [==============================] - 1s 44us/sample - loss: 0.1483 - accuracy: 0.9430 - val_loss: 0.1670 - val_accuracy: 0.9367
Epoch 39/200
22000/22000 [==============================] - 1s 44us/sample - loss: 0.1451 - accuracy: 0.945

22000/22000 [==============================] - 1s 47us/sample - loss: 0.1099 - accuracy: 0.9586 - val_loss: 0.1613 - val_accuracy: 0.9444
Epoch 88/200
22000/22000 [==============================] - 1s 48us/sample - loss: 0.1081 - accuracy: 0.9605 - val_loss: 0.1546 - val_accuracy: 0.9438
Epoch 89/200
22000/22000 [==============================] - 1s 51us/sample - loss: 0.1038 - accuracy: 0.9611 - val_loss: 0.1606 - val_accuracy: 0.9442
Epoch 90/200
22000/22000 [==============================] - 1s 46us/sample - loss: 0.1065 - accuracy: 0.9602 - val_loss: 0.1582 - val_accuracy: 0.9451
Epoch 91/200
22000/22000 [==============================] - 1s 47us/sample - loss: 0.1077 - accuracy: 0.9599 - val_loss: 0.1577 - val_accuracy: 0.9467
Epoch 92/200
22000/22000 [==============================] - 1s 47us/sample - loss: 0.1053 - accuracy: 0.9605 - val_loss: 0.1537 - val_accuracy: 0.9422
Epoch 93/200
22000/22000 [==============================] - 1s 56us/sample - loss: 0.1074 - accuracy: 0.959

22000/22000 [==============================] - 1s 44us/sample - loss: 0.0918 - accuracy: 0.9661 - val_loss: 0.1555 - val_accuracy: 0.9493
Epoch 142/200
22000/22000 [==============================] - 1s 46us/sample - loss: 0.0867 - accuracy: 0.9681 - val_loss: 0.1521 - val_accuracy: 0.9462
Epoch 143/200
22000/22000 [==============================] - 1s 45us/sample - loss: 0.0887 - accuracy: 0.9682 - val_loss: 0.1590 - val_accuracy: 0.9467
Epoch 144/200
22000/22000 [==============================] - 1s 43us/sample - loss: 0.0847 - accuracy: 0.9697 - val_loss: 0.1487 - val_accuracy: 0.9482
Epoch 145/200
22000/22000 [==============================] - 1s 44us/sample - loss: 0.0896 - accuracy: 0.9668 - val_loss: 0.1602 - val_accuracy: 0.9476
Epoch 146/200
22000/22000 [==============================] - 1s 43us/sample - loss: 0.0849 - accuracy: 0.9697 - val_loss: 0.1596 - val_accuracy: 0.9480
Epoch 147/200
22000/22000 [==============================] - 1s 44us/sample - loss: 0.0856 - accuracy:

In [47]:
pd.DataFrame(acc,index=['train','valid','test','rest data']).to_csv('./results/acc_lr_ann.csv')

,LEFT,RIGHT
train,0.987864,0.986136
valid,0.954364,0.951091
test,0.920873,0.904291
rest data,0.927044,0.894982


# test features from 2 of 8 signals

In [67]:
#test features from 2 of 8 signals

acc_2f={}
cols = [ 'LEFT_TA','LEFT_TS','LEFT_BF', 'LEFT_RF',
        'RIGHT_TA','RIGHT_TS','RIGHT_BF', 'RIGHT_RF']


for p in combinations(cols[:4],2):
    ind_temp=feature.columns.str.contains(p[0])| feature.columns.str.contains(p[1])
    feature_sc = sc.fit_transform(feature.loc[:,ind_temp])
    feature2_sc = sc.transform(feature2.loc[:,ind_temp])
    train,valid,test = train_model(model,feature_sc,np.array(y))
    acc_rest=test_model(model,feature2_sc,np.array(y2))
    acc_2f[p[0]+'_'+p[1]] = [train,valid,test,acc_rest]
    print(acc_rest)
    
for p in combinations(cols[4:],2):
    ind_temp=feature.columns.str.contains(p[0])| feature.columns.str.contains(p[1])
    feature_sc = sc.fit_transform(feature.loc[:,ind_temp])
    feature2_sc = sc.transform(feature2.loc[:,ind_temp])
    train,valid,test = train_model(model,feature_sc,np.array(y))
    acc_rest=test_model(model,feature2_sc,np.array(y2))
    acc_2f[p[0]+'_'+p[1]] = [train,valid,test,acc_rest]
    print(acc_rest)

Train on 22000 samples, validate on 5500 samples
Epoch 1/200
22000/22000 [==============================] - 2s 93us/sample - loss: 0.2918 - accuracy: 0.8795 - val_loss: 0.2951 - val_accuracy: 0.8795
Epoch 2/200
22000/22000 [==============================] - 1s 47us/sample - loss: 0.2900 - accuracy: 0.8787 - val_loss: 0.2929 - val_accuracy: 0.8813
Epoch 3/200
22000/22000 [==============================] - 1s 47us/sample - loss: 0.2927 - accuracy: 0.8792 - val_loss: 0.2951 - val_accuracy: 0.8778
Epoch 4/200
22000/22000 [==============================] - 1s 47us/sample - loss: 0.2918 - accuracy: 0.8788 - val_loss: 0.2927 - val_accuracy: 0.8804
Epoch 5/200
22000/22000 [==============================] - 1s 61us/sample - loss: 0.2853 - accuracy: 0.8810 - val_loss: 0.2954 - val_accuracy: 0.8773
Epoch 6/200
22000/22000 [==============================] - 1s 57us/sample - loss: 0.2922 - accuracy: 0.8788 - val_loss: 0.2912 - val_accuracy: 0.8793
Epoch 7/200
22000/22000 [==========================

Epoch 32/200
22000/22000 [==============================] - 1s 51us/sample - loss: 0.2643 - accuracy: 0.8957 - val_loss: 0.2539 - val_accuracy: 0.8984
Epoch 33/200
22000/22000 [==============================] - 1s 46us/sample - loss: 0.2653 - accuracy: 0.8923 - val_loss: 0.2553 - val_accuracy: 0.8984
Epoch 34/200
22000/22000 [==============================] - 1s 46us/sample - loss: 0.2624 - accuracy: 0.8951 - val_loss: 0.2546 - val_accuracy: 0.8989
Epoch 35/200
22000/22000 [==============================] - 1s 46us/sample - loss: 0.2610 - accuracy: 0.8941 - val_loss: 0.2507 - val_accuracy: 0.8989
Epoch 36/200
22000/22000 [==============================] - 1s 46us/sample - loss: 0.2653 - accuracy: 0.8940 - val_loss: 0.2509 - val_accuracy: 0.8980
Epoch 37/200
22000/22000 [==============================] - 1s 46us/sample - loss: 0.2595 - accuracy: 0.8942 - val_loss: 0.2515 - val_accuracy: 0.9020
Epoch 38/200
22000/22000 [==============================] - 1s 46us/sample - loss: 0.2605 - ac

Epoch 2/200
22000/22000 [==============================] - 1s 50us/sample - loss: 0.3814 - accuracy: 0.8343 - val_loss: 0.3601 - val_accuracy: 0.8460
Epoch 3/200
22000/22000 [==============================] - 1s 49us/sample - loss: 0.3690 - accuracy: 0.8378 - val_loss: 0.3516 - val_accuracy: 0.8525
Epoch 4/200
22000/22000 [==============================] - 1s 52us/sample - loss: 0.3555 - accuracy: 0.8460 - val_loss: 0.3409 - val_accuracy: 0.8587
Epoch 5/200
22000/22000 [==============================] - 1s 46us/sample - loss: 0.3517 - accuracy: 0.8492 - val_loss: 0.3383 - val_accuracy: 0.8555
Epoch 6/200
22000/22000 [==============================] - 1s 45us/sample - loss: 0.3486 - accuracy: 0.8501 - val_loss: 0.3328 - val_accuracy: 0.8573
Epoch 7/200
22000/22000 [==============================] - 1s 46us/sample - loss: 0.3387 - accuracy: 0.8545 - val_loss: 0.3308 - val_accuracy: 0.8602
Epoch 8/200
22000/22000 [==============================] - 1s 46us/sample - loss: 0.3397 - accuracy:

22000/22000 [==============================] - 1s 48us/sample - loss: 0.2824 - accuracy: 0.8822 - val_loss: 0.2874 - val_accuracy: 0.8791
Epoch 57/200
22000/22000 [==============================] - 1s 44us/sample - loss: 0.2788 - accuracy: 0.8834 - val_loss: 0.2890 - val_accuracy: 0.8845
Epoch 58/200
22000/22000 [==============================] - 1s 49us/sample - loss: 0.2760 - accuracy: 0.8841 - val_loss: 0.2897 - val_accuracy: 0.8805
Epoch 59/200
22000/22000 [==============================] - 1s 55us/sample - loss: 0.2759 - accuracy: 0.8855 - val_loss: 0.2874 - val_accuracy: 0.8789
Epoch 60/200
22000/22000 [==============================] - 1s 47us/sample - loss: 0.2776 - accuracy: 0.8847 - val_loss: 0.2865 - val_accuracy: 0.8782
Epoch 61/200
22000/22000 [==============================] - 1s 57us/sample - loss: 0.2760 - accuracy: 0.8849 - val_loss: 0.2832 - val_accuracy: 0.8833
Epoch 62/200
22000/22000 [==============================] - 1s 54us/sample - loss: 0.2777 - accuracy: 0.885

22000/22000 [==============================] - 1s 45us/sample - loss: 0.2885 - accuracy: 0.8839 - val_loss: 0.2788 - val_accuracy: 0.8869
Epoch 33/200
22000/22000 [==============================] - 1s 44us/sample - loss: 0.2867 - accuracy: 0.8831 - val_loss: 0.2801 - val_accuracy: 0.8856
Epoch 34/200
22000/22000 [==============================] - 1s 43us/sample - loss: 0.2852 - accuracy: 0.8864 - val_loss: 0.2775 - val_accuracy: 0.8876
Epoch 35/200
22000/22000 [==============================] - 1s 46us/sample - loss: 0.2873 - accuracy: 0.8850 - val_loss: 0.2775 - val_accuracy: 0.8853
Epoch 36/200
22000/22000 [==============================] - 1s 43us/sample - loss: 0.2847 - accuracy: 0.8878 - val_loss: 0.2754 - val_accuracy: 0.8871
Epoch 37/200
22000/22000 [==============================] - 1s 43us/sample - loss: 0.2847 - accuracy: 0.8849 - val_loss: 0.2763 - val_accuracy: 0.8858
Epoch 38/200
22000/22000 [==============================] - 1s 43us/sample - loss: 0.2848 - accuracy: 0.885

...

22000/22000 [==============================] - 1s 44us/sample - loss: 0.2755 - accuracy: 0.8909 - val_loss: 0.2647 - val_accuracy: 0.8929
Epoch 25/200
22000/22000 [==============================] - 1s 45us/sample - loss: 0.2763 - accuracy: 0.8929 - val_loss: 0.2641 - val_accuracy: 0.8964
Epoch 26/200
22000/22000 [==============================] - 1s 45us/sample - loss: 0.2717 - accuracy: 0.8935 - val_loss: 0.2618 - val_accuracy: 0.8971
Epoch 27/200
22000/22000 [==============================] - 1s 44us/sample - loss: 0.2730 - accuracy: 0.8937 - val_loss: 0.2613 - val_accuracy: 0.8960
Epoch 28/200
22000/22000 [==============================] - 1s 45us/sample - loss: 0.2734 - accuracy: 0.8899 - val_loss: 0.2587 - val_accuracy: 0.8962
Epoch 29/200
22000/22000 [==============================] - 1s 45us/sample - loss: 0.2683 - accuracy: 0.8953 - val_loss: 0.2596 - val_accuracy: 0.8951
Epoch 30/200
22000/22000 [==============================] - 1s 45us/sample - loss: 0.2672 - accuracy: 0.897

22000/22000 [==============================] - 1s 58us/sample - loss: 0.2305 - accuracy: 0.9115 - val_loss: 0.2451 - val_accuracy: 0.9067
Epoch 79/200
22000/22000 [==============================] - 1s 55us/sample - loss: 0.2322 - accuracy: 0.9095 - val_loss: 0.2470 - val_accuracy: 0.9053
Epoch 80/200
22000/22000 [==============================] - 1s 56us/sample - loss: 0.2307 - accuracy: 0.9131 - val_loss: 0.2492 - val_accuracy: 0.9056
Epoch 81/200
22000/22000 [==============================] - 1s 54us/sample - loss: 0.2340 - accuracy: 0.9101 - val_loss: 0.2423 - val_accuracy: 0.9065
Epoch 82/200
22000/22000 [==============================] - 1s 55us/sample - loss: 0.2304 - accuracy: 0.9120 - val_loss: 0.2449 - val_accuracy: 0.9073
Epoch 83/200
22000/22000 [==============================] - 1s 54us/sample - loss: 0.2278 - accuracy: 0.9133 - val_loss: 0.2460 - val_accuracy: 0.9065
Epoch 84/200
22000/22000 [==============================] - 1s 55us/sample - loss: 0.2307 - accuracy: 0.912

22000/22000 [==============================] - 1s 49us/sample - loss: 0.2609 - accuracy: 0.8990 - val_loss: 0.2586 - val_accuracy: 0.8973
Epoch 35/200
22000/22000 [==============================] - 1s 48us/sample - loss: 0.2593 - accuracy: 0.9002 - val_loss: 0.2608 - val_accuracy: 0.8951
Epoch 36/200
22000/22000 [==============================] - 1s 48us/sample - loss: 0.2596 - accuracy: 0.8978 - val_loss: 0.2565 - val_accuracy: 0.8976
Epoch 37/200
22000/22000 [==============================] - 1s 48us/sample - loss: 0.2564 - accuracy: 0.8994 - val_loss: 0.2556 - val_accuracy: 0.8998
Epoch 38/200
22000/22000 [==============================] - 1s 50us/sample - loss: 0.2591 - accuracy: 0.8985 - val_loss: 0.2567 - val_accuracy: 0.8964
Epoch 39/200
22000/22000 [==============================] - 1s 60us/sample - loss: 0.2568 - accuracy: 0.8983 - val_loss: 0.2549 - val_accuracy: 0.8991
Epoch 40/200
22000/22000 [==============================] - 1s 51us/sample - loss: 0.2510 - accuracy: 0.903

22000/22000 [==============================] - 1s 65us/sample - loss: 0.2245 - accuracy: 0.9135 - val_loss: 0.2362 - val_accuracy: 0.9100
Epoch 89/200
22000/22000 [==============================] - 1s 55us/sample - loss: 0.2243 - accuracy: 0.9154 - val_loss: 0.2380 - val_accuracy: 0.9104
Epoch 90/200
22000/22000 [==============================] - 1s 60us/sample - loss: 0.2216 - accuracy: 0.9144 - val_loss: 0.2348 - val_accuracy: 0.9080
Epoch 91/200
22000/22000 [==============================] - 1s 53us/sample - loss: 0.2267 - accuracy: 0.9135 - val_loss: 0.2366 - val_accuracy: 0.9085
Epoch 92/200
22000/22000 [==============================] - 1s 55us/sample - loss: 0.2219 - accuracy: 0.9137 - val_loss: 0.2333 - val_accuracy: 0.9111
Epoch 93/200
22000/22000 [==============================] - 1s 53us/sample - loss: 0.2235 - accuracy: 0.9151 - val_loss: 0.2367 - val_accuracy: 0.9084
Epoch 94/200
22000/22000 [==============================] - 1s 54us/sample - loss: 0.2199 - accuracy: 0.916

22000/22000 [==============================] - 1s 53us/sample - loss: 0.2054 - accuracy: 0.9224 - val_loss: 0.2336 - val_accuracy: 0.9113
Epoch 143/200
22000/22000 [==============================] - 1s 57us/sample - loss: 0.2068 - accuracy: 0.9221 - val_loss: 0.2330 - val_accuracy: 0.9115
Epoch 144/200
22000/22000 [==============================] - 1s 53us/sample - loss: 0.2034 - accuracy: 0.9221 - val_loss: 0.2308 - val_accuracy: 0.9118
test:0.857455
valid:0.913273
train:0.948091
acc:0.848029
0.8480292480538952


In [73]:
pd.DataFrame(acc_2f,index=['train','valid','test','rest data']).T

,train,valid,test,rest data
LEFT_TA_LEFT_TS,0.912545,0.881273,0.789818,0.782118
LEFT_TA_LEFT_BF,0.931636,0.906909,0.904873,0.864276
LEFT_TA_LEFT_RF,0.915045,0.884545,0.859200,0.898761
LEFT_TS_LEFT_BF,0.933955,0.901636,0.848727,0.824697
LEFT_TS_LEFT_RF,0.933045,0.884182,0.848582,0.856040
LEFT_BF_LEFT_RF,0.927545,0.898909,0.876073,0.868178
RIGHT_TA_RIGHT_TS,0.928909,0.902000,0.865164,0.850987
RIGHT_TA_RIGHT_BF,0.938591,0.906000,0.872582,0.864995
RIGHT_TA_RIGHT_RF,0.928455,0.902545,0.877818,0.857539
RIGHT_TS_RIGHT_BF,0.945182,0.906182,0.900800,0.850289


# Test on some of rest

In [219]:
some_of_rest = ['正常/P940_MSham_B_Walking_trial_6_emg.csv',
                '正常/P940_M050_B_Walking_trial_4_emg.csv',
                '正常/P812_M100_A_Walking_trial_3_emg.csv',
                '正常/P645_M050_A_Walking_trial_3_emg.csv',
                '正常/P623_Msham_B_Walking_trial_2_emg.csv',
                '正常/P551_M50_B_Walking_trial_6_emg.csv',
                'P379_M050_2_OFF_A_FoG_trial_1_emg.csv',
                'P551_M050_2_B_FoG_trial_2_emg.csv']
#booster = xgb.Booster()
#booster.load_model('./model/XGBoost_W256_S64_Left.json')
#model = xgb.XGBClassifier()
#model._Booster = booster
acc = []
columns=['LEFT_TA','LEFT_TS','LEFT_BF', 'LEFT_RF']
for file in some_of_rest:
    path = './data/'+file
    feature2,y2 = dp.pipeline_feature(path,width=256,stride=64,scaler=False,
                                      threshold_WAMP=threshold_WAMP,
                                      threshold_ZC=threshold_ZC,
                                      threshold_SSC=threshold_SSC,
                                      bins=bins,
                                      ranges=HIST_range,
                                      show_para=False,
                                      filt = 250)
    feature2_sc = sc.transform(feature2)
    acc += [test_model(model,feature2_sc,y2)]
print(acc)

acc:0.997041
acc:0.994048
acc:0.956284
acc:0.980276
acc:0.991055
acc:0.906706
acc:0.856749
acc:0.966764
[0.9970414201183432, 0.9940476190476191, 0.9562841530054644, 0.980276134122288, 0.9910554561717353, 0.9067055393586005, 0.8567493112947658, 0.9667644183773216]


In [263]:
y_pred=model.predict(feature2_sc)
metrics.accuracy_score(y2,y_pred>0.5)

0.9589472691255542

## RNN

In [149]:
feature_re = feature_sc.reshape((-1,feature.shape[1],1))
x_full,x_test,y_full,y_test = train_test_split(feature_re,y_01,test_size=0.2,random_state=123)
x_train,x_valid,y_train,y_valid = train_test_split(x_full,y_full,test_size=0.2,random_state=555)

In [156]:
input_ = layers.Input(shape=[feature.shape[1],1])
lstm1 = layers.GRU(100)(input_)
#lstm2 = layers.LSTM(20)(lstm1)
output = layers.Dense(1,activation='sigmoid')(lstm1)
model = Model(inputs=[input_],outputs=[output])

In [157]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [158]:
early_stopping = callbacks.EarlyStopping(patience = 20,
                                         monitor = 'val_accuracy', 
                                         restore_best_weights=True)
history = model.fit(x_train,y_train,validation_data=[x_valid,y_valid],
                    epochs=100,batch_size=500,
                   callbacks=[early_stopping])

Train on 21990 samples, validate on 5498 samples
Epoch 1/100
21990/21990 [==============================] - 55s 3ms/sample - loss: 0.6715 - accuracy: 0.5768 - val_loss: 0.6168 - val_accuracy: 0.6808
Epoch 2/100
21990/21990 [==============================] - 52s 2ms/sample - loss: 0.5631 - accuracy: 0.7150 - val_loss: 0.5077 - val_accuracy: 0.7552
Epoch 3/100
21990/21990 [==============================] - 52s 2ms/sample - loss: 0.5141 - accuracy: 0.7546 - val_loss: 0.4896 - val_accuracy: 0.7696
Epoch 4/100
21990/21990 [==============================] - 51s 2ms/sample - loss: 0.4999 - accuracy: 0.7643 - val_loss: 0.4811 - val_accuracy: 0.7803
Epoch 5/100
21990/21990 [==============================] - 51s 2ms/sample - loss: 0.4728 - accuracy: 0.7853 - val_loss: 0.4479 - val_accuracy: 0.7968
Epoch 6/100
21990/21990 [==============================] - 54s 2ms/sample - loss: 0.4521 - accuracy: 0.7983 - val_loss: 0.4376 - val_accuracy: 0.8032
Epoch 7/100
21990/21990 [==========================

Epoch 55/100
21990/21990 [==============================] - 59s 3ms/sample - loss: 0.2446 - accuracy: 0.9028 - val_loss: 0.2694 - val_accuracy: 0.8909
Epoch 56/100
21990/21990 [==============================] - 60s 3ms/sample - loss: 0.2435 - accuracy: 0.9028 - val_loss: 0.2641 - val_accuracy: 0.8927
Epoch 57/100
21990/21990 [==============================] - 58s 3ms/sample - loss: 0.2359 - accuracy: 0.9075 - val_loss: 0.2672 - val_accuracy: 0.8927
Epoch 58/100
21990/21990 [==============================] - 58s 3ms/sample - loss: 0.2331 - accuracy: 0.9080 - val_loss: 0.2556 - val_accuracy: 0.8994
Epoch 59/100
21990/21990 [==============================] - 66s 3ms/sample - loss: 0.2314 - accuracy: 0.9074 - val_loss: 0.2608 - val_accuracy: 0.8920
Epoch 60/100
21990/21990 [==============================] - 58s 3ms/sample - loss: 0.2337 - accuracy: 0.9083 - val_loss: 0.2617 - val_accuracy: 0.8952
Epoch 61/100
21990/21990 [==============================] - 58s 3ms/sample - loss: 0.2263 - ac

In [159]:
feature2_re = feature2_sc.reshape((-1,feature.shape[1],1))

In [160]:
test_model(model,feature2_re,y2)

acc:0.889541


0.8895405669599218